<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="200" height="200">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="200" height="200">

# Autoencoder Variable Transformation
<hr style="border-top-width: 4px; border-top-color: #34609b;"> 

## Initialize DataLoader and Factory 

In [1]:
TMVA::Tools::Instance();
TString path = "../datasets/higgs-dataset.root";
if (gSystem->AccessPathName( path ))  // file does not exist in local directory
        gSystem->Exec("curl -O http://files.oproject.org/root/tmva/datasets/higgs/higgs-dataset.root");
TFile *input = TFile::Open( path );
TMVA::DataLoader *loader2=new TMVA::DataLoader("higgs-dataset");
TTree *Tsignal     = (TTree*)input->Get("TreeS");
TTree *Tbackground = (TTree*)input->Get("TreeB");
TString outfileName( "higgs_output.root" );
TFile* outputFile = TFile::Open( outfileName, "RECREATE" );
TMVA::Factory *factory = new TMVA::Factory( "TMVAClassification", outputFile,
            "!V:!Silent:Color:DrawProgressBar:AnalysisType=Classification" );

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


## Setup DataLoader

In [2]:
loader2->AddVariable("lepton_pT",'F');
loader2->AddVariable("lepton_eta",'F');
loader2->AddVariable("lepton_phi",'F');
loader2->AddVariable("missing_energy_magnitude",'F');
loader2->AddVariable("missing_energy_phi",'F');
loader2->AddVariable("jet_1_pt",'F');
loader2->AddVariable("jet_1_eta",'F');
loader2->AddVariable("jet_1_phi",'F');
loader2->AddVariable("jet_1_b_tag",'F');
loader2->AddVariable("jet_2_pt",'F');
loader2->AddVariable("jet_2_eta",'F');
loader2->AddVariable("jet_2_phi",'F');
loader2->AddVariable("jet_2_b_tag",'F');
loader2->AddVariable("jet_3_pt",'F');
loader2->AddVariable("jet_3_eta",'F');
loader2->AddVariable("jet_3_phi",'F');
loader2->AddVariable("jet_3_b_tag",'F');
loader2->AddVariable("jet_4_pt",'F');
loader2->AddVariable("jet_4_eta",'F');
loader2->AddVariable("jet_4_phi",'F');
loader2->AddVariable("jet_4_b_tag",'F');
loader2->AddVariable("m_jj",'F');
loader2->AddVariable("m_jjj",'F');
loader2->AddVariable("m_lv",'F');
loader2->AddVariable("m_jlv",'F');
loader2->AddVariable("m_bb",'F');
loader2->AddVariable("m_wbb",'F');
loader2->AddVariable("m_wwbb",'F');

## Setup DataLoader 

In [3]:
Double_t signalWeight     = 1.0;
Double_t backgroundWeight = 1.0;
loader2->AddSignalTree    ( Tsignal,     signalWeight     );
loader2->AddBackgroundTree( Tbackground, backgroundWeight );
TCut myCuts = ""; 
TCut myCutb = "";
loader2->PrepareTrainingAndTestTree( myCuts, myCutb,
                                         "nTrain_Signal=1829123:nTrain_Background=1170877:nTest_Signal=4000000:nTest_Background=4000000:SplitMode=Random:NormMode=NumEvents:!V" );

--- DataSetInfo              : Dataset[higgs-dataset] : Added class "Signal"	 with internal class number 0
--- higgs-dataset            : Add Tree TreeS of type Signal with 5829123 events
--- DataSetInfo              : Dataset[higgs-dataset] : Added class "Background"	 with internal class number 1
--- higgs-dataset            : Add Tree TreeB of type Background with 5170877 events
--- higgs-dataset            : Preparing trees for training and testing...


## Get Dataset Info

In [4]:
// calculates min, max, mean, RMS and variance of all variables
// loader2->CalcNorm();

## Setup Neural Network

In [5]:
TString layoutString ("Layout=TANH|100,TANH|50,TANH|10,LINEAR");
TString training0 ("LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True");
TString training1 ("LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1");
TString training2 ("LearningRate=1e-2,Momentum=0.3,Repetitions=1,ConvergenceSteps=300,BatchSize=40,TestRepetitions=7,WeightDecay=0.0001,Regularization=L2,Multithreading=True");
TString training3 ("LearningRate=1e-3,Momentum=0.1,Repetitions=1,ConvergenceSteps=200,BatchSize=70,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True");

TString trainingStrategyString ("TrainingStrategy=");
trainingStrategyString += training0 + "|" + training1 + "|" + training2 + "|" + training3;
TString nnOptions ("AE(!H:V:ErrorStrategy=CROSSENTROPY:VarTransform=G:WeightInitialization=XAVIERUNIFORM");
nnOptions.Append (":");
nnOptions.Append (layoutString);
nnOptions.Append (":");
nnOptions.Append (trainingStrategyString);
nnOptions.Append (")");

In [ ]:
// print nnOptions string 
cout << nnOptions.Data() << endl;

AE(!H:V:ErrorStrategy=CROSSENTROPY:VarTransform=G:WeightInitialization=XAVIERUNIFORM:Layout=TANH|100,TANH|50,TANH|10,LINEAR:TrainingStrategy=LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True|LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1|LearningRate=1e-2,Momentum=0.3,Repetitions=1,ConvergenceSteps=300,BatchSize=40,TestRepetitions=7,WeightDecay=0.0001,Regularization=L2,Multithreading=True|LearningRate=1e-3,Momentum=0.1,Repetitions=1,ConvergenceSteps=200,BatchSize=70,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True)


## Apply Autoencoder Transform

In [ ]:
TMVA::DataLoader* newloader = loader2->VarTransform(nnOptions);

--- higgs-dataset            : Train method: DNN for Classification
--- DataSetFactory           : Dataset[higgs-dataset] : Splitmode is: "RANDOM" the mixmode is: "SAMEASSPLITMODE"
--- DataSetFactory           : Dataset[higgs-dataset] : Create training and testing trees -- looping over class "Signal" ...
--- DataSetFactory           : Dataset[higgs-dataset] : Weight expression for class 'Signal': ""
--- DataSetFactory           : Dataset[higgs-dataset] : Create training and testing trees -- looping over class "Background" ...
--- DataSetFactory           : Dataset[higgs-dataset] : Weight expression for class 'Background': ""
--- DataSetFactory           : Dataset[higgs-dataset] : Number of events in input trees (after possible flattening of arrays):
--- DataSetFactory           : Dataset[higgs-dataset] :     Signal          -- number of events       : 5829123  / sum of weights: 5.82912e+06
--- DataSetFactory           : Dataset[higgs-dataset] :     Background      -- number of events  


 *** Break *** segmentation violation


[<unknown binary>]
[<unknown binary>]
[/Users/testuser/root-build/lib/libTMVA.so] TMVA::MethodBase::TrainMethod() /Users/testuser/root-build/include/TMVA/Configurable.h:130
[/Users/testuser/root-build/lib/libTMVA.so] TMVA::DataLoader::VarTransform(TString) /Users/testuser/root-build/include/TMVA/Configurable.h:130
[<unknown binary>]
[<unknown binary>]
[<unknown binary>]
[<unknown binary>]
[<unknown binary>]
[<unknown binary>]
[<unknown binary>]
[<unknown binary>]
[<unknown binary>]
[/Users/testuser/root-build/lib/libCling.so] HandleInterpreterException(cling::MetaProcessor*, char const*, cling::Interpreter::CompilationResult&, cling::Value*) /Users/testuser/root/core/meta/src/TCling.cxx:1885
[/Users/testuser/root-build/lib/libCling.so] TCling::ProcessLine(char const*, TInterpreter::EErrorCode*) /Users/testuser/root/core/meta/src/TCling.cxx:2047
[/Users/testuser/root-build/lib/libJupyROOT.so] JupyROOTExecutorImpl(char const*) /Users/testuser/root/bindings/pyroot/JupyROOT/src/IOHandler.c